In [1]:
import numpy as np 
import pandas as pd
import time
import datetime
import gc
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from gensim.models import word2vec
import Levenshtein


In [2]:
train_dataset = pd.read_csv('../temp/train_offline_alldata_df.csv')
valid_dataset = pd.read_csv('../temp/valid_offline_alldata_df.csv')
valid_dataset_29 = pd.read_csv('../temp/valid_offline_df.csv')
valid_dataset = valid_dataset[(len(valid_dataset) - len(valid_dataset_29)):]


In [3]:
fea = [
    'query_prediction_number', 'query_prediction_max', 'query_prediction_min', 'query_prediction_mean', 'query_prediction_std',
       'prefix_count', 'prefix_rate',
 'title_count', 'title_rate', 'tag_count', 'tag_rate',
 'query_prediction_count', 'query_prediction_rate', 'prefix_title_count',
 'prefix_title_rate',  'prefix_tag_count', 'prefix_tag_rate',
 'title_tag_count', 'title_tag_rate',
    'prefix_click_number', 'title_click_number', 'query_prediction_click_number', 'prefix_tag_click_number', 
    'prefix_title_click_number', 'title_tag_click_number',
    'is_title_in_query', 'is_prefix_in_title', 
    'title_tag_types', 'prefix_tag_types', 'tag_title_types', 'tag_prefix_types',
 'title_prefix_types', 'prefix_title_types', 'tag_query_prediction_types', 'title_query_prediction_types',
      'prefix_len', 'title_len',
 'query_prediction_key_len_max', 'query_prediction_key_len_min',
 'query_prediction_key_len_mean', 'query_prediction_key_len_std',
 'len_title-prefix', 'len_prefix/title', 'len_mean-title', 'len_mean/title',
    'q_t_word_match', 'q_t_jaccard', 'q_t_common_words',
 'q_t_total_unique_words', 'q_t_wc_diff', 'q_t_wc_ratio',
 'q_t_wc_diff_unique', 'q_t_wc_ratio_unique', 'q_t_tfidf_word_match_share',
 'p_t_word_match', 'p_t_jaccard', 'p_t_common_words',
 'p_t_total_unique_words', 'p_t_wc_diff', 'p_t_wc_ratio',
 'p_t_wc_diff_unique', 'p_t_wc_ratio_unique', 'p_t_tfidf_word_match_share',
 'p_q_word_match', 'p_q_jaccard', 'p_q_common_words',
 'p_q_total_unique_words', 'p_q_wc_diff', 'p_q_wc_ratio',
 'p_q_wc_diff_unique', 'p_q_wc_ratio_unique', 'p_q_tfidf_word_match_share',
    'title_prefix_dot_similarity',
 'title_query_dot_similarity', 'title_prefix_norm_similarity',
 'title_query_norm_similarity', 'title_prefix_cosine_similarity',
 'title_query_cosine_similarity',
    'title_query_dot_similarity_max', 'title_query_dot_similarity_min',
 'title_query_dot_similarity_mean', 'title_query_dot_similarity_std',
    'title_query_norm_similarity_min', 'title_query_norm_similarity_mean',
 'title_query_norm_similarity_std', 'title_prefix_cosine_similarity',
    'title_query_cosine_similarity_max', 'title_query_cosine_similarity_min',
 'title_query_cosine_similarity_mean', 'title_query_cosine_similarity_std',
    'title_prefix_leven', 'title_prefix_leven_rate',
 'title_query_leven_sum', 'title_query_leven_max', 'title_query_leven_min',
 'title_query_leven_mean', 'title_query_leven_std',
      ]





In [4]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=127, max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.8, colsample_bytree=1, subsample_freq=1,
    learning_rate=0.01, random_state=2018, n_jobs=-1
)

valid_dataset['predicted_score'] = 0

lgb_model.fit(train_dataset[fea], train_dataset['label'], eval_set=[(train_dataset[fea], train_dataset['label']),
                            (valid_dataset[fea], valid_dataset['label'])], early_stopping_rounds=50, eval_metric='auc')
valid_pred = lgb_model.predict_proba(valid_dataset[fea], num_iteration=lgb_model.best_iteration_)[:, 1]
print(np.mean(valid_pred))

fscore = lgb_model.booster_.feature_importance()
feaNames = lgb_model.booster_.feature_name()
scoreDf = pd.DataFrame(index=feaNames, columns=['importance'], data=fscore)
print(scoreDf.sort_index(by=['importance'], ascending=False))



[1]	valid_0's auc: 0.886943	valid_1's auc: 0.872326
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.887387	valid_1's auc: 0.872914
[3]	valid_0's auc: 0.887785	valid_1's auc: 0.873679
[4]	valid_0's auc: 0.887871	valid_1's auc: 0.873383
[5]	valid_0's auc: 0.888127	valid_1's auc: 0.874478
[6]	valid_0's auc: 0.888183	valid_1's auc: 0.874684
[7]	valid_0's auc: 0.888306	valid_1's auc: 0.875259
[8]	valid_0's auc: 0.888362	valid_1's auc: 0.875296
[9]	valid_0's auc: 0.888514	valid_1's auc: 0.875533
[10]	valid_0's auc: 0.888603	valid_1's auc: 0.875772
[11]	valid_0's auc: 0.888671	valid_1's auc: 0.875953
[12]	valid_0's auc: 0.888688	valid_1's auc: 0.87576
[13]	valid_0's auc: 0.888715	valid_1's auc: 0.87617
[14]	valid_0's auc: 0.888805	valid_1's auc: 0.876044
[15]	valid_0's auc: 0.888903	valid_1's auc: 0.876236
[16]	valid_0's auc: 0.888987	valid_1's auc: 0.876251
[17]	valid_0's auc: 0.88901	valid_1's auc: 0.876517
[18]	valid_0's auc: 0.889014	valid_1's auc: 0.876

[155]	valid_0's auc: 0.892584	valid_1's auc: 0.880113
[156]	valid_0's auc: 0.892598	valid_1's auc: 0.880129
[157]	valid_0's auc: 0.892613	valid_1's auc: 0.880145
[158]	valid_0's auc: 0.892626	valid_1's auc: 0.880156
[159]	valid_0's auc: 0.89264	valid_1's auc: 0.880163
[160]	valid_0's auc: 0.892653	valid_1's auc: 0.88017
[161]	valid_0's auc: 0.892668	valid_1's auc: 0.880173
[162]	valid_0's auc: 0.892681	valid_1's auc: 0.880192
[163]	valid_0's auc: 0.892694	valid_1's auc: 0.880201
[164]	valid_0's auc: 0.892707	valid_1's auc: 0.880217
[165]	valid_0's auc: 0.892718	valid_1's auc: 0.880229
[166]	valid_0's auc: 0.892733	valid_1's auc: 0.880256
[167]	valid_0's auc: 0.892745	valid_1's auc: 0.880268
[168]	valid_0's auc: 0.892759	valid_1's auc: 0.880276
[169]	valid_0's auc: 0.892775	valid_1's auc: 0.880291
[170]	valid_0's auc: 0.892787	valid_1's auc: 0.880306
[171]	valid_0's auc: 0.892798	valid_1's auc: 0.880313
[172]	valid_0's auc: 0.892812	valid_1's auc: 0.880323
[173]	valid_0's auc: 0.892824	

[308]	valid_0's auc: 0.8944	valid_1's auc: 0.881504
[309]	valid_0's auc: 0.894408	valid_1's auc: 0.88151
[310]	valid_0's auc: 0.894417	valid_1's auc: 0.881527
[311]	valid_0's auc: 0.894425	valid_1's auc: 0.881533
[312]	valid_0's auc: 0.894434	valid_1's auc: 0.881536
[313]	valid_0's auc: 0.894442	valid_1's auc: 0.881538
[314]	valid_0's auc: 0.894451	valid_1's auc: 0.881541
[315]	valid_0's auc: 0.89446	valid_1's auc: 0.881551
[316]	valid_0's auc: 0.894468	valid_1's auc: 0.881556
[317]	valid_0's auc: 0.894476	valid_1's auc: 0.881564
[318]	valid_0's auc: 0.894485	valid_1's auc: 0.88157
[319]	valid_0's auc: 0.894495	valid_1's auc: 0.881585
[320]	valid_0's auc: 0.894503	valid_1's auc: 0.88159
[321]	valid_0's auc: 0.894512	valid_1's auc: 0.881592
[322]	valid_0's auc: 0.894522	valid_1's auc: 0.881595
[323]	valid_0's auc: 0.894531	valid_1's auc: 0.881606
[324]	valid_0's auc: 0.89454	valid_1's auc: 0.88162
[325]	valid_0's auc: 0.894548	valid_1's auc: 0.881628
[326]	valid_0's auc: 0.894557	valid_

[461]	valid_0's auc: 0.895728	valid_1's auc: 0.882526
[462]	valid_0's auc: 0.895737	valid_1's auc: 0.882526
[463]	valid_0's auc: 0.895744	valid_1's auc: 0.88254
[464]	valid_0's auc: 0.895751	valid_1's auc: 0.882544
[465]	valid_0's auc: 0.895758	valid_1's auc: 0.882547
[466]	valid_0's auc: 0.895765	valid_1's auc: 0.882556
[467]	valid_0's auc: 0.895772	valid_1's auc: 0.882566
[468]	valid_0's auc: 0.89578	valid_1's auc: 0.882579
[469]	valid_0's auc: 0.895786	valid_1's auc: 0.882583
[470]	valid_0's auc: 0.895793	valid_1's auc: 0.882585
[471]	valid_0's auc: 0.8958	valid_1's auc: 0.882602
[472]	valid_0's auc: 0.895808	valid_1's auc: 0.882601
[473]	valid_0's auc: 0.895816	valid_1's auc: 0.882606
[474]	valid_0's auc: 0.895822	valid_1's auc: 0.882609
[475]	valid_0's auc: 0.895828	valid_1's auc: 0.882612
[476]	valid_0's auc: 0.895838	valid_1's auc: 0.882616
[477]	valid_0's auc: 0.895846	valid_1's auc: 0.882622
[478]	valid_0's auc: 0.895855	valid_1's auc: 0.882624
[479]	valid_0's auc: 0.895863	va

[614]	valid_0's auc: 0.896668	valid_1's auc: 0.883107
[615]	valid_0's auc: 0.896674	valid_1's auc: 0.883111
[616]	valid_0's auc: 0.896679	valid_1's auc: 0.88312
[617]	valid_0's auc: 0.896683	valid_1's auc: 0.883121
[618]	valid_0's auc: 0.896689	valid_1's auc: 0.883122
[619]	valid_0's auc: 0.896693	valid_1's auc: 0.883119
[620]	valid_0's auc: 0.896698	valid_1's auc: 0.883113
[621]	valid_0's auc: 0.896702	valid_1's auc: 0.883116
[622]	valid_0's auc: 0.896707	valid_1's auc: 0.883118
[623]	valid_0's auc: 0.896711	valid_1's auc: 0.883125
[624]	valid_0's auc: 0.896718	valid_1's auc: 0.883122
[625]	valid_0's auc: 0.896722	valid_1's auc: 0.883123
[626]	valid_0's auc: 0.896728	valid_1's auc: 0.883146
[627]	valid_0's auc: 0.896733	valid_1's auc: 0.883138
[628]	valid_0's auc: 0.896738	valid_1's auc: 0.883142
[629]	valid_0's auc: 0.896743	valid_1's auc: 0.883147
[630]	valid_0's auc: 0.896747	valid_1's auc: 0.883157
[631]	valid_0's auc: 0.896752	valid_1's auc: 0.883156
[632]	valid_0's auc: 0.896757

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [6]:
valid_dataset['predicted_score'] = valid_pred

train_prefix_set = set(train_dataset['prefix'])

valid_dataset['is_prefix_in_train'] = valid_dataset['prefix'].map(lambda x : 1 if x in train_prefix_set else 0)
print(np.mean(valid_dataset[valid_dataset.is_prefix_in_train == 1]['predicted_score']))
print(np.mean(valid_dataset[valid_dataset.is_prefix_in_train == 0]['predicted_score']))



0.372032800736592
0.4681375893293212


In [7]:
#直接搜索最佳阈值

yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 700, 2):
    real_yuzhi = yuzhi / 1000
    valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_dataset['label'], valid_dataset['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)




{'0.35': 0.74322746658407, '0.352': 0.7434582448775134, '0.354': 0.7440015294904886, '0.356': 0.7446630088410762, '0.358': 0.7448567583157084, '0.36': 0.7449667662074944, '0.362': 0.7454549844274366, '0.364': 0.745688091148794, '0.366': 0.7456829646876211, '0.368': 0.7459159859976662, '0.37': 0.7464009938372348, '0.372': 0.7465298142717497, '0.374': 0.7469808686279794, '0.376': 0.7473601493050439, '0.378': 0.7474245814176481, '0.38': 0.7473225496713198, '0.382': 0.7475879086983345, '0.384': 0.7474577529257491, '0.386': 0.7480445956347925, '0.388': 0.7484699208837139, '0.39': 0.7487166708198354, '0.392': 0.7479459580950477, '0.394': 0.7479540506043998, '0.396': 0.7476930792377132, '0.398': 0.7478959929657079, '0.4': 0.7481446074115072, '0.402': 0.7478506416559514, '0.404': 0.7478546188793539, '0.406': 0.7479576093279713, '0.408': 0.7477842491770068, '0.41': 0.7478128565994675, '0.412': 0.7480939310765476, '0.414': 0.7481981408378964, '0.416': 0.7480806998750222, '0.418': 0.7477332516026

In [19]:
valid_prefix0_df = valid_dataset[valid_dataset.is_prefix_in_train == 1].copy()

#定义调整函数
def resultAdjustment(result_df, t):
    result_df_temp = result_df.copy()
    result_df_temp['x'] = result_df_temp.predicted_score.map(lambda x: -(math.log(((1 - x) / x), math.e)))
    result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t)))) 
    print(result_df_temp['adjust_result'].mean())
    return result_df_temp['adjust_result']

print('original mean : ', valid_prefix0_df['predicted_score'].mean())
valid_df_after = resultAdjustment(valid_prefix0_df, 0.74105)



original mean :  0.372032800736592
0.46814736059864015


In [20]:
valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 1] = valid_df_after
print(np.mean(valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 1]))




0.4681375893293212
0.46814736059864015


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 700, 2):
    real_yuzhi = yuzhi / 1000
    valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_dataset['label'], valid_dataset['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)



{'0.35': 0.7233666989559788, '0.352': 0.7240263941001425, '0.354': 0.7246214003326924, '0.356': 0.7253513501808181, '0.358': 0.7261979789199174, '0.36': 0.7267577401573089, '0.362': 0.7274434858578137, '0.364': 0.7283515424868084, '0.366': 0.7284820546592031, '0.368': 0.7289057168644199, '0.37': 0.7295622450104752, '0.372': 0.7301846779181762, '0.374': 0.7306892125182596, '0.376': 0.7315168190290227, '0.378': 0.7320360280217948, '0.38': 0.7321158129175946, '0.382': 0.7326715002788623, '0.384': 0.7333705980994969, '0.386': 0.7345116863308062, '0.388': 0.7352359550561798, '0.39': 0.7360241452317672, '0.392': 0.7358418165812698, '0.394': 0.7362063115032237, '0.396': 0.7367275694790769, '0.398': 0.7369257308879608, '0.4': 0.737612843077063, '0.402': 0.7378388837039902, '0.404': 0.7379964433906343, '0.406': 0.7383356943746286, '0.408': 0.7386678260073679, '0.41': 0.7388961500538854, '0.412': 0.7395000459516587, '0.414': 0.7399907961343763, '0.416': 0.740097241744821, '0.418': 0.740108432345

In [19]:
valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > 0.526 else 0)



In [20]:
print(np.mean(valid_dataset['label'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['label'][valid_dataset.is_prefix_in_train == 1]))
print(np.mean(valid_dataset['predicted_label'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['predicted_label'][valid_dataset.is_prefix_in_train == 1]))



0.37339522022516297
0.37126949892160305
0.4296859569425242
0.42521442544013643
